In [4]:
!pip install --quiet jupyterlab-vim jupytex
!jupyter labextension enable

# Neo4j

In [5]:
!pip install --quiet neo4j

In [13]:
#!curl http://neo4j:7474
#!curl http://neo4j:7687

In [6]:
from neo4j import GraphDatabase, RoutingControl

#URI = "neo4j://localhost:7687"
#URI = "bolt://localhost:7687"
#URI = "bolt://127.0.0.1:7687"
URI = "bolt://neo4j:7687"
AUTH = ("neo4j", "testtest")

def add_friend(driver, name, friend_name):
    driver.execute_query(
        "MERGE (a:Person {name: $name}) "
        "MERGE (friend:Person {name: $friend_name}) "
        "MERGE (a)-[:KNOWS]->(friend)",
        name=name, friend_name=friend_name, database_="neo4j",
    )


def print_friends(driver, name):
    records, _, _ = driver.execute_query(
        "MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
        "RETURN friend.name ORDER BY friend.name",
        name=name, database_="neo4j", routing_=RoutingControl.READ,
    )
    for record in records:
        print(record["friend.name"])


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    add_friend(driver, "Arthur", "Guinevere")
    add_friend(driver, "Arthur", "Lancelot")
    add_friend(driver, "Arthur", "Merlin")
    print_friends(driver, "Arthur")

Guinevere
Lancelot
Merlin


# Example

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)

- Every application using Neo4j needs a `driver` object
- A `driver` object holds the details to the connection to a Neo4j database (e.g., URIs, credentials, and configuration).

In [113]:
# Test the connection to the DB without executing any query.
driver.verify_connectivity()

In [114]:
driver.get_server_info()

In [115]:
# `execute_query()` wraps lower level APIs (e.g., Sessions) and it's used for simple cases.
query = "MATCH(n) RETURN COUNT(n) AS node_count"
query_result = driver.execute_query(query)

In [116]:
print("type=", type(query_result))
print(query_result)

type= <class 'neo4j._work.eager_result.EagerResult'>
EagerResult(records=[<Record node_count=1>], summary=<neo4j._work.summary.ResultSummary object at 0xffff3f657250>, keys=['node_count'])


In [117]:
results, summary, keys = query_result
print(records)
print(summary)
print(keys)

[<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>]
['node_count']


## Session

- Database activity is coordinated through `Session`s and `Transaction`s
- A `Session` is a container for a number of unit of works
    - Provide guarantees of causal consistency
    - Are lightweight opeation and not thread safe
- A `Transaction` is a unit of work that is either committed in its entirety or rolled back in case of failure

In [105]:
# Create `Session`.
session = driver.session(database="neo4j")
print("session.closed()=", session.closed())

# Run a query.
query = "MATCH (n) RETURN n"
_ = session.run(query)

# Close `Session`.
session.close()
print("session.closed()=", session.closed())

session.closed()= False
session.closed()= True


In [99]:
# Session can be created and destroyed using a block context, so that the session is closed
# properly in case of exceptions.
with driver.session() as session:
    result = session.run("MATCH (n) RETURN n")
    # ...

In [118]:
# Create `Session`.
session = driver.session(database="neo4j")
print("session.closed()=", session.closed())

session.closed()= False


In [62]:
# Count the number of nodes.
query = "MATCH(n) RETURN COUNT(n) AS node_count"
node_count_result = session.run(query)
node_count = node_count_result.single()["node_count"]
print(node_count)

1


In [61]:
# Count the number of relations.
query = "MATCH ()-[r]->() RETURN COUNT(r) AS relationship_count;"
result = session.run(query)
rel_count = result.single()["relationship_count"]
print(rel_count)

0


In [70]:
query = "MATCH (n) RETURN n"
records, summary, keys = driver.execute_query(query)
print(records)
print(summary)
print(keys)

[<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>]
['n']


In [71]:
for record in records:
    print(record.data())

{'n': {'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}}


In [75]:
print(summary)

In [73]:
# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

The query `MATCH (n) RETURN n` returned 1 records in 27 ms.


In [76]:
print(records[0])

<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>


In [77]:
print(records[0].data())

{'n': {'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}}


In [52]:
# node_count_result = driver.execute_query(
#     "MATCH(n) RETURN COUNT(n) AS node_count")
# node_count = node_count_result["node_count"]

TypeError: tuple indices must be integers or slices, not str

In [14]:
# Delete everything.
driver.execute_query(
    "MATCH(n) OPTIONAL MATCH (n)-[r]-() DELETE n, r"
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff64fc9c00>, keys=[])

In [ ]:
driver.execute_query(
    "MERGE (a:Person {name: $name}) "
    "MERGE (friend:Person {name: $friend_name}) "
    "MERGE (a)-[:KNOWS]->(friend)",
    name=name, friend_name=friend_name, database_="neo4j",
)

In [ ]:
session.close()
driver.close()

## Plot

In [58]:
query = 'CREATE (w:Wine {name:"Prancing Wolf", style: "ice wine", vintage: 2015})'
driver.execute_query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff3f48e950>, keys=[])

In [8]:
!pip install --quiet networkx matplotlib pyvis

In [13]:
from neo4j import GraphDatabase
from pyvis.network import Network

uri = URI
AUTH = ("neo4j", "testtest")
user, password = AUTH

# Define a function to fetch nodes and relationships from Neo4j
def fetch_graph_data(uri, user, password, query):
    driver = GraphDatabase.driver(URI, auth=AUTH)
    nodes = []
    relationships = []
    
    with driver.session() as session:
        results = session.run(query)
        
        for record in results:
            n = record['n']
            m = record['m']
            r = record['r']
            
            nodes.append((n.id, dict(n)))
            nodes.append((m.id, dict(m)))
            relationships.append((n.id, m.id, r.type))
    
    driver.close()
    return nodes, relationships

# Define the query to fetch nodes and relationships
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 100
"""

# Fetch the graph data
nodes, relationships = fetch_graph_data(uri, user, password, query)

# Create a Pyvis Network graph
net = Network(notebook=True)

# Add nodes to the Pyvis graph
for node_id, node_data in nodes:
    net.add_node(node_id, label=node_data.get('name', str(node_id)))

# Add edges to the Pyvis graph
for source, target, label in relationships:
    net.add_edge(source, target, label=label)

# Show the graph
net.show("graph.html")

graph.html


/tmp/ipykernel_2254/2436837932.py:22: DeprecationWarning: `id` is deprecated, use `element_id` instead
  nodes.append((n.id, dict(n)))
/tmp/ipykernel_2254/2436837932.py:23: DeprecationWarning: `id` is deprecated, use `element_id` instead
  nodes.append((m.id, dict(m)))
/tmp/ipykernel_2254/2436837932.py:24: DeprecationWarning: `id` is deprecated, use `element_id` instead
  relationships.append((n.id, m.id, r.type))


# pyvis

In [23]:
!pip install --quiet pyvis

In [36]:
from pyvis.network import Network

# Sample Cypher query to retrieve nodes and relationships
cypher_query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 10
"""
# Execute the query and fetch the results

with driver.session() as session:
  result = session.run(cypher_query)

  net = Network(cdn_resources = "remote", directed = True, height = '500px',width = '100%',
                  notebook = True)

  for record in result:
    #print(result)
    node_a = record["n"]
    node_b = record["m"]
    relationship = record["r"]

    #add nodes
    net.add_node(node_a.element_id)
    net.add_node(node_b.element_id)
    net.add_edge(node_a.element_id, node_b.element_id)

#save html format
#net.show(file_name,notebook=False)

net.show("ex.html")

ex.html


/tmp/ipykernel_35/186928212.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
